In [1]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install opendatasets
!pip install pandas

In [3]:
import os  
import matplotlib.pyplot as plt  
import numpy as np 
import matplotlib.image as mpimg  
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import tensorflow as tf 
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  
from tensorflow.keras.callbacks import ModelCheckpoint  
import warnings

warnings.filterwarnings("ignore")


tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow successfully updated to version:", tf.__version__)

TensorFlow successfully updated to version: 2.13.0


In [4]:
dataset_path = r"/kaggle/input/plantvillage/dataset"

In [5]:
os.environ['KAGGLE_CONFIG_DIR']='C:/Users/ADMIN/Desktop/Major project/kaggle.json'

In [6]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

Traceback (most recent call last):
  File "c:\users\sahil\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\sahil\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\SAHIL\anaconda3\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "c:\users\sahil\anaconda3\lib\site-packages\kaggle\__init__.py", line 19, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "c:\users\sahil\anaconda3\lib\site-packages\kaggle\api\__init__.py", line 22, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "c:\users\sahil\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 274, in <module>
    class KaggleApi(KaggleApi):
  File "c:\users\sahil\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 296, in KaggleApi
    os.makedirs(config_dir)
  File "c:\users\sahil\anaconda3\lib\os.py", line 213, in makedirs
   

In [7]:
data_dir = 'archive/New Plant Diseases Dataset(Augmented)'
train_dir =  'archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = 'archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_dir = 'archive/test/test'

In [8]:
import tensorflow as tf
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [9]:
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)
valid_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)


In [10]:
batch_size = 32
image_size = (224, 224)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 69768 images belonging to 38 classes.
Found 17452 images belonging to 38 classes.
Found 0 images belonging to 0 classes.


In [12]:
base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(224, 224, 3))


In [13]:
base_model.trainable = False

In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(38, activation='sigmoid')(x)

In [15]:
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  
              metrics=['accuracy'])

In [17]:
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
checkpoint_filepath = 'efficientnetv2s_PLANT_DISEASE.h5'
mc = ModelCheckpoint(filepath = checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True)
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, verbose=1)

In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[es, mc, lr]
)

print('\nRestoring best Weights for MobileNetV2')
model.load_weights(checkpoint_filepath)

Epoch 1/5
2181/2181 [==============================] - ETA: 0s - loss: 0.0312 - accuracy: 0.8494
Epoch 1: val_accuracy improved from -inf to 0.94476, saving model to efficientnetv2s_PLANT_DISEASE.h5
2181/2181 [==============================] - 6331s 3s/step - loss: 0.0312 - accuracy: 0.8494 - val_loss: 0.0123 - val_accuracy: 0.9448 - lr: 0.0010
Epoch 2/5
2181/2181 [==============================] - ETA: 0s - loss: 0.0132 - accuracy: 0.9350
Epoch 2: val_accuracy improved from 0.94476 to 0.95691, saving model to efficientnetv2s_PLANT_DISEASE.h5
2181/2181 [==============================] - 4824s 2s/step - loss: 0.0132 - accuracy: 0.9350 - val_loss: 0.0093 - val_accuracy: 0.9569 - lr: 0.0010
Epoch 3/5
2181/2181 [==============================] - ETA: 0s - loss: 0.0104 - accuracy: 0.9492
Epoch 3: val_accuracy improved from 0.95691 to 0.95743, saving model to efficientnetv2s_PLANT_DISEASE.h5
2181/2181 [==============================] - 4831s 2s/step - loss: 0.0104 - accuracy: 0.9492 - val_lo

In [3]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

train_dir = r'archive\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train'
valid_dir = r'archive\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid'

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

model = Sequential()
model.add(EfficientNetV2S(include_top=False, input_shape=(224, 224, 3), weights='imagenet'))
model.add(GlobalAveragePooling2D())
model.add(Dense(train_generator.num_classes, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


checkpoint_path = 'path/to/checkpoints/efficientnetv2.h5'
checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

epochs = 10
for epoch in range(epochs):
    model.fit(train_generator,
              epochs=1, 
              validation_data=valid_generator,
              callbacks=[checkpoint_callback])

    loss, accuracy = model.evaluate(valid_generator)
    print(f'Validation accuracy after epoch {epoch + 1}: {accuracy:.2f}')

    if os.path.exists(checkpoint_path):
        model.load_weights(checkpoint_path)
    else:
        print("Checkpoint file not found.")

    
    if epoch > 0:
        if accuracy <= prev_accuracy:
            print("Validation accuracy did not improve. Stopping training.")
            break

    prev_accuracy = accuracy


if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)


loss, accuracy = model.evaluate(valid_generator)
print(f'Final validation accuracy: {accuracy:.2f}')


X_train = np.concatenate([batch[0] for batch in train_generator], axis=0)
y_train = np.concatenate([batch[1] for batch in train_generator], axis=0)
X_valid = np.concatenate([batch[0] for batch in valid_generator], axis=0)
y_valid = np.concatenate([batch[1] for batch in valid_generator], axis=0)

rf_model = RandomForestClassifier()
rf_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

rf_accuracy = rf_model.score(X_valid.reshape(X_valid.shape[0], -1), y_valid)
print(f'Random Forest accuracy: {rf_accuracy:.2f}')

ImportError: cannot import name 'layers' from partially initialized module 'keras.src' (most likely due to a circular import) (C:\Users\SAHIL\anaconda3\lib\site-packages\keras\src\__init__.py)

In [19]:
pwd

'C:\\Users\\SAHIL\\Downloads\\Major project\\Major project'

In [21]:
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve, accuracy_score, classification_report, confusion_matrix
corr_pred = metrics.confusion_matrix(valid_generator, )

n_correct = np.int((corr_pred[0][0] + corr_pred[1][1]))
print('> Correct Predictions:', n_correct)
n_wrongs = np.int((corr_pred[0][1] + (corr_pred[1][0])))
print('> Wrong Predictions:', n_wrongs)

sns.heatmap(corr_pred,annot=True, fmt="d",cmap="Blues")
plt.show()

print(metrics.classification_report(y_test, preds, 
                           target_names=["NonViolence", "Violence"]))

TypeError: confusion_matrix() missing 1 required positional argument: 'y_pred'